In [1]:
import numpy as np
import pandas as pd
import os
from human_eval.data import write_jsonl, read_problems, extract_python

In [2]:
df = pd.read_json("../data/samples_codellama.jsonl", lines=True)
df["full_response"]

0      Here is the complete code:\n```python\nimport ...
1      Plan:\n\n1. Import the necessary libraries for...
2      [PYTHON]\nimport numpy as np\nfrom skimage imp...
3      Here is a possible implementation of the bland...
4      Here's the code for the function `combine_colu...
                             ...                        
565    Here is the complete code for the function `wo...
566    Plan:\n\n1. Convert the input image to graysca...
567    Here is the complete code for the function `wo...
568    ```python\nimport numpy as np\nimport pandas a...
569    Here is the complete code for the function `wo...
Name: full_response, Length: 570, dtype: object

In [3]:
directory = "../data/"

collection = []
for filename in os.listdir(directory):
    jsonl_file = filename.endswith(".jsonl") and not filename.endswith("results.jsonl") and not filename.endswith("bia.jsonl")
    results_jsonl_file = filename.endswith("_results.jsonl")
    if jsonl_file:
        df = pd.read_json(directory + filename, lines=True)
        df['model'] = filename.replace(".jsonl", "").replace("samples_", "")
        collection.append(df)
    # if results_jsonl_file:
    #     df = pd.read_json(directory + filename, lines=True)
    #     df['model'] = filename.replace("_results.jsonl", "").replace("samples_", "")
    #     collection.append(df)

df = pd.concat(collection)


In [4]:
df

,task_id,completion,full_response,model
0,../test_cases/apply_otsu_threshold_and_count_p...,\n# Plan:\n# 1. Import required libraries (cv2...,Here's the completed code with a step-by-step ...,claude-3-5-sonnet-20240620
1,../test_cases/binary_closing.ipynb,\n# Plan:\n# 1. Import necessary libraries (nu...,"Here's the completed code with a plan, necessa...",claude-3-5-sonnet-20240620
2,../test_cases/binary_skeleton.ipynb,\n# Plan:\n# 1. Import necessary libraries (sk...,Here's the completed code with a step-by-step ...,claude-3-5-sonnet-20240620
3,../test_cases/bland_altman.ipynb,\n# Plan:\n# 1. Import required libraries (pan...,"Here's the completed code with a plan, necessa...",claude-3-5-sonnet-20240620
4,../test_cases/combine_columns_of_tables.ipynb,\n# Plan:\n# 1. Import required library (panda...,Here's the completed code with a step-by-step ...,claude-3-5-sonnet-20240620
...,...,...,...,...
565,../test_cases/workflow_batch_process_folder_me...,\n import os\n import pandas as pd\n ...,\n import os\n import pandas as pd\n ...,reference
566,../test_cases/workflow_segmentation_counting.i...,\n import skimage\n import numpy as np\n...,\n import skimage\n import numpy as np\n...,reference
567,../test_cases/workflow_segmentation_measuremen...,\n import skimage\n import numpy as np\n...,\n import skimage\n import numpy as np\n...,reference
568,../test_cases/workflow_segment_measure_umap.ipynb,\n import numpy as np\n import pandas as...,\n import numpy as np\n import pandas as...,reference


In [5]:
test = """
hallo
```
code
```
bla
```
code2
```
"""
extract_python(test)

'\ncode\n\ncode2\n'

In [6]:
# Function-mapping to extract python code from full_response
output_dir = directory # overwrite input
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

df["completion"] = df["full_response"].astype(str).apply(extract_python)
grouped = df.groupby("model")

for name, group in grouped:
    records = group.drop("model", axis=1).to_dict("records")
    write_jsonl(f"{output_dir}/samples_{name}.jsonl", records)

# Testing code after 
for filename in os.listdir(directory):
    jsonl_file = filename.endswith(".jsonl") and not filename.endswith("results.jsonl") and not filename.endswith("bia.jsonl")
    results_jsonl_file = filename.endswith("_results.jsonl")
    if jsonl_file:
        model_id = filename.split(".jsonl")[0]
        df_orig = pd.read_json(directory+filename, lines=True)
        df_new = pd.read_json(f"{output_dir}{model_id}.jsonl", lines=True)
        
        if not df_orig.equals(df_new):
            # checks inequalities
            num_diff = np.size(np.where(~np.all((df_orig == df_new).to_numpy(), axis=1)))
            print(f"{model_id} is different in {num_diff} entries")
        